This notebook was copied from VADIM KAMAEV's notebook (https://www.kaggle.com/code/vadimkamaev/icr-identify-age), I would also like to give a shout-out to SAMUEL's idea of using Tabpfn transformer on this dataset (https://www.kaggle.com/code/muelsamu/simple-tabpfn-approach-for-score-of-15-in-1-min/input). Also i am using validation from this amazing notebook (by ERTUĞRUL DEMIR) : https://www.kaggle.com/code/datafan07/icr-simple-eda-baseline

In [1]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages
Processing /kaggle/input/pip-packages-icr/pip-packages/tabpfn-0.1.9-py3-none-any.whl


In [2]:
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [3]:
# Define competition's metric (from ERTUĞRUL DEMIR's notebook)
def balanced_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred / np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    
    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / 2
    
    return logloss

In [4]:
import numpy as np
import pandas as pd
import json


from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
import xgboost
from tabpfn import TabPFNClassifier
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# loading data and performing EDA

In [5]:
# LOAD THE DATA

BASE_DIR = '/kaggle/input/icr-identify-age-related-conditions'
# Import data directly as H2O frame
maindf = pd.read_csv(f'{BASE_DIR}/train.csv')
greeksdf = pd.read_csv(f'{BASE_DIR}/greeks.csv')
testdf = pd.read_csv(f'{BASE_DIR}/test.csv')
# train_int_df
# = pd.read_csv('/kaggle/input/icr-integer-data/train_integerized.csv')

print(maindf.EJ.unique())
first_cat = maindf.EJ.unique()[0]
maindf.EJ = maindf.EJ.eq(first_cat).astype('int')
testdf.EJ = testdf.EJ.eq(first_cat).astype('int')
# train_int_df.EJ = train_int_df.EJ.eq(first_cat).astype('int')

['B' 'A']


In [6]:
# train_int_df

In [7]:
# maindf.describe()

In [8]:
# train_int_df.describe() 

In [9]:
# Greeks contains time information that we can use, we just need to parse it to int / nan.

from datetime import date, datetime
times = greeksdf.Epsilon.copy()
times[greeksdf.Epsilon != 'Unknown'] = greeksdf.Epsilon[greeksdf.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
times[greeksdf.Epsilon == 'Unknown'] = np.nan

In [10]:
# opt_params = {'subsample': 0.7,
#  'n_estimators': 500,
#  'max_depth': 6,
#  'learning_rate': 0.1,
#  'colsample_bytree': 0.8999999999999999,
#  'colsample_bylevel': 0.7,
#  'scale_pos_weight': 10}

opt_params = {'n_estimators': 2000, 
              'max_depth': 4, 
              'colsample_bytree': 0.7, 
              'learning_rate': 0.007241565529695428, 
              'reg_lambda': 2}


In [11]:
class WeightedEns(BaseEstimator):
    def __init__(self):
        self.classifiers = [xgboost.XGBClassifier(**opt_params),TabPFNClassifier(N_ensemble_configurations=64,device='cuda:0')]
#         self.classifiers = [xgboost.XGBClassifier(**opt_params)]
        self.imp = SimpleImputer(missing_values=np.nan, strategy='median')
    
#     def fit(self, X, y):
#         cls, y = np.unique(y, return_inverse=True)
#         self.classes_ = cls
#         X = self.imp.fit_transform(X)
#         for cl in self.classifiers:
#             cl.fit(X,y)

    def fit(self, X, y):
        cls, y = np.unique(y, return_inverse=True)
        self.classes_ = cls
        X = self.imp.fit_transform(X)

        trained_classifiers = []
        for cl in self.classifiers:
            cl.fit(X, y)
            trained_classifiers.append(cl)

        return trained_classifiers
    
    def predict_proba(self, X):
        X = self.imp.transform(X)
        ps = np.stack([cl.predict_proba(X) for cl in self.classifiers])
        p = np.mean(ps,axis=0)
        class_0_est_instances = p[:,0].sum()
        others_est_instances = p[:,1:].sum()
        # we reweight the probs, since the loss is also balanced like this
        # our models out of the box optimize CE
        # with these changes they optimize balanced CE
        new_p = p * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(p.shape[1])]])
        return new_p / np.sum(new_p,axis=1,keepdims=1)

# Feature Engineering

In [12]:
# maindf['GL'] = 0.05/(maindf['GL']+0.05)
# testdf['GL'] = 0.05/(testdf['GL']+0.05)


In [13]:
# Set predictor and target columns
target = 'Class'
predictors = [n for n in maindf.columns if n != target and n not in ["Id"] ] # remove target and unnecessary features

In [14]:
pred_and_time = pd.concat((maindf[predictors], times), 1)

In [15]:
test_predictors = np.array(testdf[predictors])
test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors),1)) + pred_and_time.Epsilon.max()+1),1)

# Validation

In [16]:
import sys
sys.path.append('/kaggle/input/iterativestratification')
from scipy import stats
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split ,ParameterGrid ,ParameterSampler,learning_curve,LearningCurveDisplay ,StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [17]:
# import sklearn
# sklearn.metrics.get_scorer_names()

In [18]:
m = WeightedEns()
# m = xgboost.XGBClassifier(enable_categorical = True)
oof = np.zeros((len(maindf), 2))

skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

final_preds = []

df = pd.concat((maindf, times), 1)

for train_index,val_index in skf.split(df, greeksdf.iloc[:,1:-1]):
    X_train, X_val = df.loc[train_index, predictors + ['Epsilon'] ], df.loc[val_index, predictors + ['Epsilon'] ]
    y_train, y_val = df.loc[train_index, 'Class'], df.loc[val_index, 'Class']
    
    m.fit(X_train,y_train)
    preds = m.predict_proba(X_val)
    
    oof[val_index, :] = preds
print("CV : " ,balanced_log_loss(maindf['Class'], oof))

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
CV :  0.19386140826302245


In [19]:
#Sanity Check
def balance_loglossv2(y_true, y_pred):
    target_mean = y_true.mean()
    w0 = 1/(1-target_mean)
    w1 = 1/target_mean
    sample_weight = [w0 if y == 0 else w1 for y in y_true]
    loss = log_loss(y_true, y_pred, sample_weight=sample_weight)
    
    return loss

print('balanced log loss v2 : ',balance_loglossv2(maindf['Class'], oof))  
print('log loss : ', log_loss(maindf['Class'], oof))
print('accuracy: ', accuracy_score(maindf['Class'], np.round(oof[:, 1])))

balanced log loss v2 :  0.19386140826302245
log loss :  0.19075009872044071
accuracy:  0.93354943273906


**Now let's check what happens when we apply the threshold :**

In [20]:
m = WeightedEns()

oof = np.zeros((len(maindf), 2))

skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

final_preds = []

df = pd.concat((maindf, times), 1)

for train_index,val_index in skf.split(df, greeksdf.iloc[:,1:-1]):

    X_train, X_val = df.loc[train_index, predictors + ['Epsilon'] ], df.loc[val_index, predictors + ['Epsilon'] ]
    y_train, y_val = df.loc[train_index, 'Class'], df.loc[val_index, 'Class']

    m.fit(X_train,y_train)
    preds = m.predict_proba(X_val)

    preds[:,:1][preds[:,:1] > 0.84] = 1
    preds[:,:1][preds[:,:1] < 0.14] = 0

    preds[:,1:][preds[:,1:] > 0.84] = 1
    preds[:,1:][preds[:,1:] < 0.14] = 0


    oof[val_index, :] = preds
print("CV : " ,balanced_log_loss(maindf['Class'], oof))


Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_24/611376446.py:9: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df = pd.concat((maindf, times), 1)


CV :  0.42340951657205184


In [21]:
#Sanity Check
def balance_loglossv2(y_true, y_pred):
    target_mean = y_true.mean()
    w0 = 1/(1-target_mean)
    w1 = 1/target_mean
    sample_weight = [w0 if y == 0 else w1 for y in y_true]
    loss = log_loss(y_true, y_pred, sample_weight=sample_weight)
    
    return loss

print('balanced log loss v2 : ',balance_loglossv2(maindf['Class'], oof))  
print('log loss : ', log_loss(maindf['Class'], oof))
print('accuracy: ', accuracy_score(maindf['Class'], np.round(oof[:, 1])))

balanced log loss v2 :  0.4377376996237792
log loss :  0.4300266140516434
accuracy:  0.93354943273906


**Notice how badly the log loss is affected by applying the threshold , we can hence deduce that this postprocessing is a very risky approach and might not generalize well to the private LB , please refer to this discussion for more info :**  https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/414209

# Hyperparmeter Tuning

In [22]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
# from sklearn.multioutput import MultiOutputClassifier

In [23]:
# # instantiate labelencoder object
# from sklearn.preprocessing import LabelEncoder
# categorical_cols=['Alpha','Beta','Gamma','Delta']
# le = LabelEncoder()
# le_df = greeksdf.iloc[:,1:-1]
# # apply le on categorical feature columns
# le_df[categorical_cols] = le_df[categorical_cols].apply(lambda col: le.fit_transform(col))

In [24]:
# df = pd.concat((maindf, times), 1)
# # df = SimpleImputer(missing_values=np.nan, strategy='median').fit_transform(df)
# df = df.fillna(df.median())

In [25]:
# X = df.loc[:, predictors + ['Epsilon'] ]
# y = df.loc[:,'Class']

In [26]:
# X.shape, y.shape

In [27]:
# # Perform hyperparameter tuning using GridSearchCV
# from sklearn.metrics import make_scorer
# print('Performing hyperparameter tuning using GridSearchCV...')
# params = {'max_depth': [3, 6, 10, 15],
#           'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
#           'subsample': np.arange(0.5, 1.0, 0.1),
#           'colsample_bytree': np.arange(0.5, 1.0, 0.1),
#           'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
#           'n_estimators': [100, 250, 500, 750],
#           }

# _cv = StratifiedKFold(n_splits=5, shuffle = True,random_state=42)

# xgb_model = xgboost.XGBClassifier(random_state=42)
# grid_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=params, scoring='roc_auc', n_jobs=-1, cv=_cv, error_score= 'raise')
# # grid_search.fit(X,y)
# # scoring=make_scorer(balanced_log_loss,greater_is_better= False)

In [28]:
# grid_search.fit(X,y)

In [29]:
# best_params = grid_search.best_params_

In [30]:
# best_params

In [31]:
# best_params = {'subsample': 0.7999999999999999,
#  'n_estimators': 250,
#  'max_depth': 6,
#  'learning_rate': 0.1,
#  'colsample_bytree': 0.7,
#  'colsample_bylevel': 0.7}


# xgb_cw = xgboost.XGBClassifier(**best_params)

# # define grid
# weights = [1, 10, 25, 50, 75, 99, 100, 1000]
# param_grid = dict(scale_pos_weight=weights)
# # define evaluation procedure
# _cv = StratifiedKFold(n_splits=5, shuffle = True,random_state=42)
# # define grid search
# grid = GridSearchCV(estimator=xgb_cw, param_grid=param_grid, n_jobs=-1, cv=_cv, scoring='roc_auc')
# # execute the grid search
# grid_result = grid.fit(X, y)
# # report the best configuration
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# # report all configurations
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [32]:
# grid_search.best_score_

In [33]:
# from skleabest_score_rn.metrics import make_scorer

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# random_search = RandomizedSearchCV(xgb, param_distributions=params, scoring=make_scorer(balanced_log_loss,greater_is_better= False) , n_jobs=-1, cv=5, verbose=0, random_state=42)
# random_search.fit(df.loc[:, predictors + ['Epsilon'] ], df.loc[:,'Class'])

In [34]:
# random_search.fit(df.loc[:, predictors + ['Epsilon'] ], df.loc[:,'Class'])

In [35]:
# random_search.best_params_

In [36]:
# def xgb_optimizer(trial, X, y, K):
#     n_estimators = trial.suggest_categorical('n_estimators', [500, 1000, 2000])
#     max_depth = trial.suggest_int('max_depth', 4, 10)
#     colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8])
#     learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-2)
#     reg_lambda = trial.suggest_categorical('reg_lambda', [0.1, 0.5, 1, 2])
    
    
#     model = xgboost.XGBClassifier(n_estimators=n_estimators,
#                           max_depth=max_depth,
#                           colsample_bytree=colsample_bytree,
#                           learning_rate=learning_rate,
#                           reg_lambda=reg_lambda)
# #                          scale_pos_weight=4.71)  ## we set class imbalance by using sampling method.
    
    
#     folds = StratifiedKFold(n_splits=K)
#     losses = []
    
#     for train_idx, val_idx in folds.split(X, y):
#         X_train = X.iloc[train_idx, :]
#         y_train = y.iloc[train_idx]
        
#         X_val = X.iloc[val_idx, :]
#         y_val = y.iloc[val_idx]
        
#         model.fit(X_train, y_train)
#         preds = model.predict_proba(X_val)
#         loss = evaluation_metric(y_val, preds)
#         losses.append(loss)
    
    
#     return np.mean(losses)

In [37]:
# # set configs
# is_tuning = True
# is_scaling = True
# is_pca = False
# apply_vif = False
# sampling_method = 'hybrid' # 'under' or 'over'
# if is_tuning:
#     n_trials=30
    
# # Keras model compile
# learning_rate = 1e-2
# batch_size = 32
# epochs = 10

In [38]:
# import random
# import os
# import tensorflow as tf


# def seed_everything(seed: int = 42):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)
#     tf.random.set_seed(seed)
    
# seed_everything()

In [39]:
# evaluation_metric = balanced_log_loss

In [40]:
# from functools import partial
# import optuna

# K = 4
# opt_func = partial(xgb_optimizer, X=X, y=y, K=K)

# if is_tuning:
#     xgb_study = optuna.create_study(direction="minimize")
#     xgb_study.optimize(opt_func, n_trials=n_trials)

In [41]:
# # visualize experiment logs
# def display_experiment_log(study):
#     display(study.trials_dataframe())
#     print("Best Score: %.4f" % study.best_value)
#     print("Best params: ", study.best_trial.params)
#     history = study.trials_dataframe()
#     display(history[history.value == study.best_value])
#     optuna.visualization.plot_optimization_history(study).show()
#     optuna.visualization.plot_param_importances(study).show()

In [42]:
# if is_tuning:
#     display_experiment_log(xgb_study)

# submit

In [43]:
m = WeightedEns()
m.fit(np.array(pred_and_time),np.array(greeksdf['Alpha']))
p = m.predict_proba(test_pred_and_time)
print(p)
assert (m.classes_[0] == "A")
p = np.concatenate((p[:,:1],np.sum(p[:,1:],1,keepdims=True)), 1)

p0 = p[:,:1]
# p0[p0 > 0.86] = 1
# p0[p0 < 0.14] = 0
submit=pd.DataFrame(testdf["Id"], columns=["Id"])
submit["class_0"] = p0
submit["class_1"] = 1 - p0
submit.to_csv('submission.csv',index=False)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
[[0.5        0.16075467 0.14661128 0.19263405]
 [0.5        0.16075467 0.14661128 0.19263405]
 [0.5        0.16075467 0.14661128 0.19263405]
 [0.5        0.16075467 0.14661128 0.19263405]
 [0.5        0.16075467 0.14661128 0.19263405]]


In [44]:
pd.read_csv('submission.csv')

,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5
